# practice_31_08

## 1. pydantic why

In [6]:
from pydantic import BaseModel, EmailStr, AnyUrl, Field
from typing import List, Dict, Optional, Annotated

class Patient(BaseModel):

    name: Annotated[
        str,
        Field(
            max_length=50,
            title="Name of the patient",
            description="Give the name of the patients in less than 50 chars",
            examples=[{"example_1": "Ramesh"}, {"example_2": "Suresh"}]
        )
    ]
    email: EmailStr
    linkedIn: Optional[AnyUrl] = None
    age: int = Field(gt=0, lt=120)
    weight: Annotated[float, Field(gt=0, strict=True)]
    married: Annotated[bool, Field(default=None, description="Is the patient married or not?")]
    allergies: Annotated[Optional[List[str]], Field(default=None, max_length=5)]
    contact_details: Optional[Dict[str, str]] = {}


def insert_patient_data(patient: Patient):
    print(patient)
    print("Data inserted!")


def update_patient_data(patient: Patient):
    print(patient)
    print("Data updated!")


patient_info = {
    "name": "John",
    "email": "john319@gmail.com",
    "linkedIn": "https://www.linkedin.com/in/john/",
    "age": 30,
    "weight": 70.5,
    "married": True,
    "contact_details": {"phone": "123-456-7890"},
}

patient1 = Patient(**patient_info)

insert_patient_data(patient1)
update_patient_data(patient1)

name='John' email='john319@gmail.com' linkedIn=AnyUrl('https://www.linkedin.com/in/john/') age=30 weight=70.5 married=True allergies=None contact_details={'phone': '123-456-7890'}
Data inserted!
name='John' email='john319@gmail.com' linkedIn=AnyUrl('https://www.linkedin.com/in/john/') age=30 weight=70.5 married=True allergies=None contact_details={'phone': '123-456-7890'}
Data updated!


## 2. Field Validators

In [2]:
from pydantic import BaseModel, EmailStr, field_validator, model_validator
from typing import List, Dict


class Patient(BaseModel):
    name: str
    email: EmailStr
    age: int
    allergies: List[str]
    contact_details: Dict[str, str]

    @field_validator('name')
    @classmethod
    def transform_name(cls, value):
        return value.upper()

    @field_validator('age', mode='after')
    @classmethod
    def transform_age(cls, value):
        if 0 < value < 100:
            return value
        else:
            raise ValueError('Invalid age')

    @field_validator('email')
    @classmethod
    def email_validator(cls, value):
        valid_domains = ["hdfc.com", "icici.com", "sbi.com"]

        domain_name = value.split('@')[-1]

        if domain_name not in valid_domains:
            raise ValueError("Invalid domain name")

        return value


def insert_patient_data(patient: Patient):
    print(patient)
    print("Data inserted!")


def update_patient_data(patient: Patient):
    print(patient)
    print("Data updated!")


patient_info = {
    "name": "John",
    "email": "john319@icici.com",
    "age": "70",
    "weight": 70.5,
    "married": True,
    "allergies": ["peanut", "milk"],
    "contact_details": {"phone": "123-456-7890", "emergency": "987-654-3210"},
}

patient1 = Patient(**patient_info)

insert_patient_data(patient1)
update_patient_data(patient1)

name='JOHN' email='john319@icici.com' age=70 allergies=['peanut', 'milk'] contact_details={'phone': '123-456-7890', 'emergency': '987-654-3210'}
Data inserted!
name='JOHN' email='john319@icici.com' age=70 allergies=['peanut', 'milk'] contact_details={'phone': '123-456-7890', 'emergency': '987-654-3210'}
Data updated!


## 3. Model validator

In [1]:
from pydantic import BaseModel, EmailStr, field_validator, model_validator
from typing import List, Dict

# schema
class Patient(BaseModel):
    name: str
    email: EmailStr
    age: int
    allergies: List[str]
    contact_details: Dict[str, str]

    @field_validator("email")
    @classmethod
    def email_validator(cls, value):
        valid_domains = ["hdfc.com", "icici.com", "sbi.com"]
        domain_name = value.split("@")[-1]
        if domain_name not in valid_domains:
            raise ValueError("Invalid domain name")

        return value

    @field_validator("name")
    @classmethod
    def transform_name(cls, value):
        return value.upper()

    @field_validator("age", mode="after")
    @classmethod
    def transform_age(cls, value):
        if 0 < value < 100:
            return value
        else:
            raise ValueError("Invalid age")

    @model_validator(mode="after")
    def validate_emergency_contact(cls, model):
        if model.age > 60 and "emergency" not in model.contact_details:
            raise ValueError("patient is older than 60 must have an emergency contact")

        return model


def insert_patient_data(patient: Patient):
    print(patient)
    print("Data inserted!")

def update_patient_data(patient: Patient):
    print(patient)
    print("Data updated!")

patient_info = {
    "name": "John",
    "email": "john319@icici.com",
    "age": "70",
    "weight": 70.5,
    "married": True,
    "allergies": ["peanut", "milk"],
    "contact_details": {"phone": "123-456-7890", "emergency": "987-654-3210"},
}

patient1 = Patient(**patient_info)

insert_patient_data(patient1)
update_patient_data(patient1)

name='JOHN' email='john319@icici.com' age=70 allergies=['peanut', 'milk'] contact_details={'phone': '123-456-7890', 'emergency': '987-654-3210'}
Data inserted!
name='JOHN' email='john319@icici.com' age=70 allergies=['peanut', 'milk'] contact_details={'phone': '123-456-7890', 'emergency': '987-654-3210'}
Data updated!


## 4. compound fields

In [ ]:
from pydantic import BaseModel, EmailStr, computed_field
from typing import List, Dict


# schema
class Patient(BaseModel):

    name: str
    email: EmailStr
    age: int
    weight: float
    height: float
    married: bool
    allergies: List[str]
    contact_details: Dict[str, str]

    # compound field
    @computed_field
    @property
    def bmi(self) -> float:
        return self.weight / (self.height**2)


def insert_patient_data(patient: Patient):
    print(patient)
    print("Data inserted!")


def update_patient_data(patient: Patient):
    print(patient)
    print("Data updated!")


patient_info = {
    "name": "John",
    "email": "john319@icici.com",
    "age": "70",
    "weight": 70.5,
    "height": 1,
    "married": True,
    "allergies": ["peanut", "milk"],
    "contact_details": {"phone": "123-456-7890"},
}

patient1 = Patient(**patient_info)

insert_patient_data(patient1)
update_patient_data(patient1)

name='John' email='john319@icici.com' age=70 weight=70.5 height=1.0 married=True allergies=['peanut', 'milk'] contact_details={'phone': '123-456-7890'} bmi=70.5
Data inserted!
name='John' email='john319@icici.com' age=70 weight=70.5 height=1.0 married=True allergies=['peanut', 'milk'] contact_details={'phone': '123-456-7890'} bmi=70.5
Data updated!


## 5. Nested models

In [5]:
from pydantic import BaseModel


class Address(BaseModel):
    city: str
    state: str
    pin: str


class Patient(BaseModel):
    name: str
    gender: str = "male"
    age: int
    address: Address


address_dict = {"city": "Gurgaon", "state": "Haryana", "pin": "122001"}
address1 = Address(**address_dict)

patient_dict = {"name": "nitish", "age": 35, "address": address1}
patient1 = Patient(**patient_dict)
print(patient1)
print(patient1.address.city)


temp = patient1.model_dump_json(exclude_unset=True)

print(temp)
print(type(temp))

name='nitish' gender='male' age=35 address=Address(city='Gurgaon', state='Haryana', pin='122001')
Gurgaon
{"name":"nitish","age":35,"address":{"city":"Gurgaon","state":"Haryana","pin":"122001"}}
<class 'str'>


# practice_19/09

In [ ]:
from pydantic import BaseModel, EmailStr, AnyUrl, Field
from typing import List, Dict, Optional, Annotated


class Patient(BaseModel):

    name: Annotated[str, Field(max_length=50, title='Name of the patient', description='Give the name of the patient', examples=['Nitish', 'Amit'])]
    email: EmailStr
    linkedin_url: AnyUrl
    age: int
    weight: Annotated[float, Field(gt=0,strict=True)]
    married: Optional[bool] = None
    allergies: Annotated[Optional[List[str]], Field(default=None, max_length=5)]
    contact_details: Dict[str, str]


def insert_patient_data(patient: Patient):

    print(patient.name)
    print(patient.age)
    print('inserted')


def update_patient_data(patient: Patient):

    print(patient.name)
    print(patient.age)
    print("inserted")


# patient_info = {'name': 'Nitish', 'age': '30', 'weight': 75.2, 'married': True, 'allergies': ['pollen', 'dust'], 'contact_details':{'email': 'abc@gmail.com', 'phone': '2345623'}}

# # optional fields
# patient_info = {
#     "name": "Nitish",
#     "age": "30",
#     "weight": 75.2,
#     "contact_details": {"email": "abc@gmail.com", "phone": "2345623"},
# }

patient_info = {
    "name": "Nitish",
    "email": "abc@gmail.com",
    "linkedin_url": "https://linkedin.com/1322",
    "age": "30",
    "weight": 75.2,
    "married": True,
    "allergies": ["pollen", "dust"],
    "contact_details": {"phone": "2345623"},
}

patient = Patient(**patient_info)

insert_patient_data(patient)
update_patient_data(patient)

Nitish
30
inserted
Nitish
30
inserted


In [ ]:
# field validator
from pydantic import BaseModel, EmailStr, AnyUrl, Field, field_validator
from typing import List, Dict, Optional, Annotated


class Patient(BaseModel):

    name: str
    email: EmailStr
    linkedin_url: AnyUrl
    age: int
    weight: Annotated[float, Field(gt=0, strict=True)]
    married: Optional[bool] = None
    allergies: Annotated[Optional[List[str]], Field(default=None, max_length=5)]
    contact_details: Dict[str, str]

    @field_validator('email')
    @classmethod
    def email_validator(cls, value):

        valid_domains = ['hdfc.com', 'icici.com']
        # abc@gmail.com
        domain_name = value.split('@')[-1]

        if domain_name not in valid_domains:
            raise ValueError('Not a valid domain')
        
        return value
    
    @field_validator('name')
    @classmethod
    def transform_name(cls, value):
        return value.upper()
    

    @field_validator('age', mode='after')
    @classmethod
    def age_validate(cls, value):

        if 0 < value < 100:
            return value
        else:
            raise ValueError("Age should be greater than 0 and less than 100.")




patient_info = {
    "name": "Nitish",
    "email": "abc@icici.com",
    "linkedin_url": "https://linkedin.com/1322",
    "age": "30",
    "weight": 75.2,
    "married": True,
    "allergies": ["pollen", "dust"],
    "contact_details": {"phone": "2345623"},
}

patient = Patient(**patient_info)

In [21]:
# field validator
from pydantic import BaseModel, EmailStr, AnyUrl, Field, model_validator
from typing import List, Dict, Optional, Annotated


class Patient(BaseModel):

    name: str
    email: EmailStr
    linkedin_url: AnyUrl
    age: int
    weight: Annotated[float, Field(gt=0, strict=True)]
    married: Optional[bool] = None
    allergies: Annotated[Optional[List[str]], Field(default=None, max_length=5)]
    contact_details: Dict[str, str]

    @model_validator(mode='after')
    def validate_emergency_contact(cls, model):
        if model.age > 60 and 'emergency' not in model.contact_details:
            raise ValueError('Patients older than 60 must have an emergency contact.')
        return model

patient_info = {
    "name": "Nitish",
    "email": "abc@icici.com",
    "linkedin_url": "https://linkedin.com/1322",
    "age": "65",
    "weight": 75.2,
    "married": True,
    "allergies": ["pollen", "dust"],
    "contact_details": {"phone": "2345623", 'emergency': '124324534'},
}

patient = Patient(**patient_info)

In [26]:
# field validator
from pydantic import BaseModel, EmailStr, AnyUrl, Field, computed_field
from typing import List, Dict, Optional, Annotated


class Patient(BaseModel):

    name: str
    email: EmailStr
    linkedin_url: AnyUrl
    age: int
    weight: Annotated[float, Field(gt=0, strict=True)]
    height: float
    married: Optional[bool] = None
    allergies: Annotated[Optional[List[str]], Field(default=None, max_length=5)]
    contact_details: Dict[str, str]

    @computed_field
    @property
    def bmi(self) -> float:
        return round(self.weight / self.height**2, 2)


patient_info = {
    "name": "Nitish",
    "email": "abc@icici.com",
    "linkedin_url": "https://linkedin.com/1322",
    "age": "65",
    "weight": 75.2,
    'height': 1.6,
    "married": True,
    "allergies": ["pollen", "dust"],
    "contact_details": {"phone": "2345623", "emergency": "124324534"},
}

patient = Patient(**patient_info)


def update_patient_data(patient: Patient):

    print(patient.bmi)
    print("inserted")

update_patient_data(patient)

29.37
inserted


In [35]:
# Nested Model

from pydantic import BaseModel


class Address(BaseModel):
    street: str
    city: str
    state: str
    pincode: int


class Patient(BaseModel):
    name: str
    gender: str
    age: int
    address: Address


address_dict = {'street' : 'Jawahar Road', 'city': 'Gurgaon', 'state': 'Haryana', 'pincode': '431223'}
patient_dict = {'name': 'Nitish', 'gender': 'male', 'age': 35, 'address': address_dict}

patient1 = Patient(**patient_dict)

print(patient1)

temp = patient1.model_dump()

temp1 = patient1.model_dump(include=["name"])
print(temp1)


temp1 = patient1.model_dump(exclude=["name"])
print(temp1)

print(type(temp))

name='Nitish' gender='male' age=35 address=Address(street='Jawahar Road', city='Gurgaon', state='Haryana', pincode=431223)
{'name': 'Nitish'}
{'gender': 'male', 'age': 35, 'address': {'street': 'Jawahar Road', 'city': 'Gurgaon', 'state': 'Haryana', 'pincode': 431223}}
<class 'dict'>
